In [1]:
# 관계식을 생성.
# 입력 → 출력에 대한 함수 → 직선방정식

# - 회귀 : 값의 흐름(값을 예측)
# - 이항분류 : 확률(입력값의 결과가 1이될 확률, 확률이 0.5보다 크면 1, 아니면 0)
# - 다항분류 : 확률(입력값의 결과가 해당 분류값이 될 확률, 각 종류별 확률 반환 - 모든 확률의 합이 1)
#   - a : 0.2, b : 0.5, c : 0.3

# 작업순서
# 1. 데이터 로딩
import seaborn as sns
ti = sns.load_dataset('titanic')

In [2]:
# 2. 타입확인
ti.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [3]:
ti.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [16]:
# 3. 독립, 종속 변수 구분
X = ti[['pclass', 'age', 'sibsp', 'parch', 'fare']]
y = ti['survived']

In [17]:
# 4. 독립변수의 결측치 제거
X.isna().sum()

pclass      0
age       177
sibsp       0
parch       0
fare        0
dtype: int64

In [18]:
# age의 결측치는 age의 평균으로 채움
X['age'] = X['age'].fillna(X['age'].mean())
X.isna().sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


pclass    0
age       0
sibsp     0
parch     0
fare      0
dtype: int64

In [19]:
# X를 표준화 시킴. 각 독립변수들은 독립적인 정보를 가짐.
# 모든 독랍변수들이 같은 정보를 표현할때엔 정규화 (MinMaxScaler)

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_ss = ss.fit_transform(X)

In [20]:
# 훈련, 테스트 데이터 분리
from sklearn.model_selection import train_test_split
x_tr, x_te, y_tr, y_te = train_test_split(
    X_ss, y, random_state=0, stratify = y
)

x_tr.shape, x_te.shape, y_tr.shape, y_te.shape

((668, 5), (223, 5), (668,), (223,))

In [21]:
# 머신러닝(이항분류)
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000).fit(x_tr, y_tr)
model.score(x_tr, y_tr), model.score(x_te, y_te)

(0.7125748502994012, 0.6860986547085202)

In [26]:
# 딥러닝(이항분류 - 생존여부)
import tensorflow as tf

model = tf.keras.Sequential()
model.add(
    tf.keras.layers.Dense(10, activation='relu', input_shape=(x_tr.shape[1],))
)
model.add(
    tf.keras.layers.Dense(1, activation='sigmoid'),
)

In [27]:
# 1개정답과 예측값의 차이: 오차
# 모든입력값의 오차의 합: 비용(cost, loss)
# 정답을 찾는 방법 정의
model.compile(
    loss="binary_crossentropy", #회귀: mse, 다항:(sparse_)categorical_c...
    optimizer="adam", #기울기 미분,가중치 업데이트 자동화 도구
    metrics=["acc"]
)

In [28]:
# 조기종료 콜백
cb = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=3)
# 모델 훈련
model.fit(x_tr,y_tr,epochs=100, verbose=1, callbacks=[cb])

Epoch 1/100
21/21 [==============================] - 1s 2ms/step - loss: 0.7112 - acc: 0.4207
Epoch 2/100
21/21 [==============================] - 0s 2ms/step - loss: 0.6967 - acc: 0.4790
Epoch 3/100
21/21 [==============================] - 0s 1ms/step - loss: 0.6847 - acc: 0.5913
Epoch 4/100
21/21 [==============================] - 0s 2ms/step - loss: 0.6730 - acc: 0.6407
Epoch 5/100
21/21 [==============================] - 0s 2ms/step - loss: 0.6625 - acc: 0.6437
Epoch 6/100
21/21 [==============================] - 0s 2ms/step - loss: 0.6518 - acc: 0.6557
Epoch 7/100
21/21 [==============================] - 0s 2ms/step - loss: 0.6418 - acc: 0.6647
Epoch 8/100
21/21 [==============================] - 0s 2ms/step - loss: 0.6317 - acc: 0.6722
Epoch 9/100
21/21 [==============================] - 0s 2ms/step - loss: 0.6226 - acc: 0.6722
Epoch 10/100
21/21 [==============================] - 0s 2ms/step - loss: 0.6137 - acc: 0.6811
Epoch 11/100
21/21 [==============================] - 0s 2m

In [29]:
	
# 성능측정
model.evaluate(x_tr, y_tr), model.evaluate(x_te, y_te)

7/7 [==============================] - 0s 5ms/step - loss: 0.5888 - acc: 0.6906


([0.541286051273346, 0.7410179376602173],
 [0.5887981653213501, 0.6905829310417175])